In [1]:
import pandas as pd
import os
import logging
import sys

### Load environment vars and directories

In [2]:
KEY_DIR = os.path.join(os.getenv("DOCUMENTS"), "govuk-network-data", "key")
KEY_PATH = os.path.join(KEY_DIR, os.listdir(KEY_DIR)[0])
PROJECT_ID = "govuk-bigquery-analytics"

#### Logging for `pandas_gbq`

In [3]:
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

### Extract page-hit only user journeys for February 11 to 18
8.8 GB

In [4]:
query = """SELECT
  COUNT(*) AS Occurrences,
  PageSeq_Length,
  PageSequence
FROM (
  SELECT
    *
  FROM (
    SELECT
      CONCAT(fullVisitorId,"-",CAST(visitId AS STRING),"-",CAST(visitNumber AS STRING)) AS sessionId,
      STRING_AGG(IF(htype = 'PAGE',
          pagePath,
          NULL),">>") OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSequence,
      SUM(IF(htype='PAGE',
          1,
          0)) OVER (PARTITION BY fullVisitorId, visitId, visitStartTime ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSeq_Length
    FROM (
      SELECT
        fullVisitorId,
        visitId,
        visitNumber,
        visitStartTime,
        hits.page.pagePath AS pagePath,
        hits.hitNumber AS hitNumber,
        hits.type AS htype
      FROM
        `govuk-bigquery-analytics.87773428.ga_sessions_*` AS sessions
      CROSS JOIN
        UNNEST(sessions.hits) AS hits 
       WHERE _TABLE_SUFFIX BETWEEN '20190110' AND '20190128' ) )
  WHERE
    PageSeq_Length >1
  GROUP BY
    sessionId,
    PageSequence,
    PageSeq_Length)
GROUP BY
  PageSequence,
  PageSeq_Length"""

### Extract data from BigQuery

In [5]:
df_in = pd.read_gbq(query,
                       project_id=PROJECT_ID,
                       reauth=False,
                       private_key=KEY_PATH,
                       dialect="standard")

Requesting query... 
Query running...
Job ID: 8d4740ca-1e76-4812-9e8f-b217fe5c8de9
  Elapsed 7.31 s. Waiting...
  Elapsed 8.62 s. Waiting...
  Elapsed 9.91 s. Waiting...
  Elapsed 11.22 s. Waiting...
  Elapsed 12.52 s. Waiting...
  Elapsed 13.91 s. Waiting...
  Elapsed 15.2 s. Waiting...
  Elapsed 16.5 s. Waiting...
  Elapsed 17.8 s. Waiting...
  Elapsed 19.09 s. Waiting...
  Elapsed 20.38 s. Waiting...
  Elapsed 21.66 s. Waiting...
  Elapsed 22.93 s. Waiting...
  Elapsed 24.22 s. Waiting...
  Elapsed 25.52 s. Waiting...
  Elapsed 26.86 s. Waiting...
  Elapsed 28.18 s. Waiting...
  Elapsed 29.46 s. Waiting...
  Elapsed 30.73 s. Waiting...
  Elapsed 32.02 s. Waiting...
  Elapsed 33.32 s. Waiting...
  Elapsed 34.6 s. Waiting...
  Elapsed 35.91 s. Waiting...
  Elapsed 37.26 s. Waiting...
  Elapsed 38.54 s. Waiting...
  Elapsed 39.84 s. Waiting...
  Elapsed 41.12 s. Waiting...
  Elapsed 42.41 s. Waiting...
  Elapsed 43.77 s. Waiting...
  Elapsed 45.06 s. Waiting...
  Elapsed 46.36 s. Waiti

In [6]:
df_in.shape

(13968085, 3)

In [7]:
df_in.head()

,Occurrences,PageSeq_Length,PageSequence
0,1,8,/budgeting-help-benefits>>/budgeting-help-bene...
1,1,33,/support-for-mortgage-interest>>/government/pu...
2,1,7,/log-in-register-hmrc-online-services>>/search...
3,1,5,/government/organisations/companies-house>>/ge...
4,1,9,/government/news/number-of-2-year-olds-eligibl...


### Explore occurrences stats

In [8]:
df_in.Occurrences.describe()

count    1.396808e+07
mean     2.766656e+00
std      3.970094e+02
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.122761e+06
Name: Occurrences, dtype: float64

In [9]:
all_occ = df_in.Occurrences.sum()
num_one_off_journeys = df_in[df_in.Occurrences==1].shape[0]
one_off_occ = df_in[df_in.Occurrences==1].Occurrences.sum()

In [10]:
all_occ, num_one_off_journeys, one_off_occ

(38644889, 12579331, 12579331)

In [11]:
df_in.sort_values("Occurrences", ascending=False).head()

,Occurrences,PageSeq_Length,PageSequence
6651719,1122761,2,/government/organisations/companies-house>>/ge...
12702836,524897,2,/government/organisations/hm-revenue-customs>>...
813759,433269,2,/universal-credit>>/sign-in-universal-credit
5165683,318898,2,/sign-in-universal-credit>>/sign-in-universal-...
13272106,240900,3,/self-assessment-tax-returns>>/log-in-file-sel...


### Add `Page_List` column

In [12]:
pagelist = [pageseq.split(">>") for pageseq in df_in['PageSequence'].values]
df_in['Page_List'] = pagelist

In [13]:
df_in['Page_List'].head()

0    [/budgeting-help-benefits, /budgeting-help-ben...
1    [/support-for-mortgage-interest, /government/p...
2    [/log-in-register-hmrc-online-services, /searc...
3    [/government/organisations/companies-house, /g...
4    [/government/news/number-of-2-year-olds-eligib...
Name: Page_List, dtype: object

In [14]:
page_views = {}
for tup in df_in.itertuples():
    for p in tup.Page_List:
        if p in page_views.keys():
            page_views[p] += tup.Occurrences
        else:
            page_views[p] = tup.Occurrences

In [15]:
len(page_views), sum(page_views.values())

(4355402, 189524646)

### Save out 

In [16]:
date_range = "jan_10_28"
bq_dir = os.path.join(os.getenv("DATA_DIR"),"raw", "bq_journey_extract")
bq_file = os.path.join(bq_dir, "pageseq_user_journey_"+date_range+".csv.gz")
bq_file_doo = os.path.join(bq_dir, "pageseq_user_journey_"+date_range+"_doo.csv.gz")
page_views_file = os.path.join(bq_dir, "pageviews_"+date_range+".csv.gz")

In [17]:
df_in.to_csv(bq_file, compression="gzip", sep='\t', index=False)

In [18]:
df_in[df_in.Occurrences>1].to_csv(bq_file_doo, compression="gzip", sep='\t', index=False)

In [19]:
import gzip
with gzip.open(page_views_file, "wb") as writer:
    writer.write("page_url\tviews\n".encode())
    for key,value in page_views.items():
        writer.write("{}\t{}\n".format(key,value).encode())